In [58]:
import concurrent.futures
import requests
import re 
import os
import pandas as pd
from bs4 import BeautifulSoup

In [112]:
#2001
base_url='https://wybory2001.pkw.gov.pl/'

main_page_url = base_url+'sjg2_k.html'

page = requests.get(main_page_url)
#page.encoding = 'utf-8'

soup = BeautifulSoup(page.content, "html.parser")
pg_txt = page.text.replace('³','ł').replace('£','Ł').replace('¶','ś').replace('±','ą').replace('ê','ę').replace('æ','ć').replace('¿','ż').replace('¦','ś').replace('ñ','ń')
pg_txt

'<html><head><title>Państwowa Komisja Wyborcza: Wybory Parlamentarne 2001</title><link href="s.css" type="text/css" rel="stylesheet"></head><body bgcolor="#EFC98B" leftmargin="0" topmargin="0" marginwidth="0" marginheight="0"><div align="center"><table width="100%" cellpadding="0" cellspacing="0" border="0" background="img/bg_top_table.gif"><tr valign=top><td width="150" background="img/bg_top_table.gif"><a href="index.html" alt="Strona główna"><img src="img/pkw_logo_left.gif" width="150" height="100" border="0"></a></td><td width="100%" background="img/bg_top_table.gif"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr valign="top"><td width="50%" background="img/bg_top_line.gif"><img src="img/p.gif" width="1" height="11"></td><td width="50%" background="img/bg_top_line.gif"><img src="img/p.gif" width="1" height="11"></td></tr><tr valign="top"><td width="50%" background="sjg2_k.html"><img src="img/pkw_napis.gif" width="330" height="54"></td><td width="50%" background=

In [103]:
path_0 = 'wyniki_wyborow/2001/'

#>Komitet Wyborczy Wyborcw "Niemiecka Mniejszo Grnego lska"</a></td><td align="right">8024</td>
wojew = re.findall(r'(Komitet[\w\"\- ]*)</a></td><td align="right">([0-9]*)</td>', pg_txt)
df_0 = pd.DataFrame(wojew, columns =['Komitet', 'Glosy']) 
df_0

,Komitet,Glosy
0,Komitet Wyborczy Sojusz Lewicy Demokratycznej ...,5342519
1,Komitet Wyborczy - Akcja Wyborcza Solidarność ...,729207
2,Komitet Wyborczy Unii Wolności,404074
3,Komitet Wyborczy Samoobrona Rzeczypospolitej P...,1327624
4,"Komitet Wyborczy ""Prawo i Sprawiedliwość""",1236787
5,Komitet Wyborczy Polskiego Stronnictwa Ludowego,1168659
6,Komitet Wyborczy Wyborców Platforma Obywatelska,1651099
7,Komitet Wyborczy Alternatywa Ruch Społeczny,54266
8,Komitet Wyborczy Polskiej Wspólnoty Narodowej,2644
9,Komitet Wyborczy Liga Polskich Rodzin,1025148


In [104]:
df_0.to_csv(path_0+'Polska.csv')

In [105]:
#geting the subpages and names of districts
#<a href="040000/sjg2_w.html">kujawsko-pomorskie</a>
#<a href="100000/sjg2_w.html">dzkie</a>
jednostki = re.findall(r'<a href="([0-9]{6}/sjg2_w.html)">([\w\-]*)</a>', pg_txt)
jednostki

[('020000/sjg2_w.html', 'dolnośląskie'),
 ('040000/sjg2_w.html', 'kujawsko-pomorskie'),
 ('060000/sjg2_w.html', 'lubelskie'),
 ('080000/sjg2_w.html', 'lubuskie'),
 ('100000/sjg2_w.html', 'łódzkie'),
 ('120000/sjg2_w.html', 'małopolskie'),
 ('140000/sjg2_w.html', 'mazowieckie'),
 ('160000/sjg2_w.html', 'opolskie'),
 ('180000/sjg2_w.html', 'podkarpackie'),
 ('200000/sjg2_w.html', 'podlaskie'),
 ('220000/sjg2_w.html', 'pomorskie'),
 ('240000/sjg2_w.html', 'śląskie'),
 ('260000/sjg2_w.html', 'świętokrzyskie'),
 ('280000/sjg2_w.html', 'warmińsko-mazurskie'),
 ('300000/sjg2_w.html', 'wielkopolskie'),
 ('320000/sjg2_w.html', 'zachodniopomorskie')]

In [ ]:
def process_district(jednostka):
    path = path_0+jednostka[1].replace(" ", "_")
    try: os.mkdir(path)
    except: pass
    
    subpage_url = base_url+jednostka[0]
    subpage = requests.get(subpage_url)
    subpage.encoding = 'utf-8'
    pg_txt = subpage.text.replace('³','ł').replace('¶','ś').replace('±','ą').replace('ê','ę').replace('æ','ć').replace('¿','ż').replace('¦','ś').replace('ñ','ń')
    #the same as previously
    komitety = re.findall(r'(Komitet[\w\"\- ]*)</a></td><td align="right">([0-9]*)</td>', pg_txt)
    df = pd.DataFrame(wojew, columns =['Komitet', 'Glosy']) 
    
    woj = jednostka[1].replace(" ", "_")

    df.to_csv(path+'/'+woj+'.csv')
    
    #<a href="SSSS03/sjg2_o.html">3 WROCŁAW</a>
    powiaty = re.findall(r'<td class="col5al"><a href="(.*.htm)" class="link1">([\w\- ]+)[\w\,\.\- ]*?</a></td>', pg_txt)
    
    for powiat in powiaty:
        subpath = path+'/'+powiat[1]
        try: os.mkdir(subpath)
        except: pass
        
        subsubpage_url = base_url+powiat[0]
        subsubpage = requests.get(subsubpage_url)
        subsubpage.encoding = 'utf-8'
        
        komitety_2 = re.findall(r'<td class="col4al"><a href=".*.htm" class="link1">([\w\- ]+)</a></td><td class="col5ar">([0-9]*)</td>', subsubpage.text)
        df_2 = pd.DataFrame(komitety_2, columns =['Komitet', 'Glosy']) 
        df_2.to_csv(subpath+'/'+powiat[1]+'.csv')
        
        #<td class="col5al"><a href="020102.htm" class="link1">Bolesławiec, gm.</a></td>
        gminy = re.findall(r'<td class="col5al"><a href="([0-9]*.htm)" CLASS="link1">([\w\,\.\- ]+)*[\w\,\.\- ]+</a></td>', subsubpage.text)
        
        for gmina in gminy:
            subsubpath = subpath+'/'+gmina[1]
            try: os.mkdir(subsubpath)
            except: pass
            
            subsubsubpage_url = base_url+gmina[0]
            subsubsubpage = requests.get(subsubsubpage_url)
            subsubsubpage.encoding = 'utf-8'
            
            komitety_3 = re.findall(r'<td class="col4al"><a href=".*.htm" class="link1">([\w\- ]+)</a></td><td class="col5ar">([0-9]*)</td>', subsubsubpage.text)
            df_3 = pd.DataFrame(komitety_3, columns =['Komitet', 'Glosy']) 
            df_3.to_csv(subsubpath+'/'+gmina[1]+'.csv')
        
    print(jednostka[1])

In [ ]:
with concurrent.futures.ProcessPoolExecutor() as executor:
    executor.map(process_district, jednostki)